<a href="https://colab.research.google.com/github/harshitt018/Advance-Data-Science/blob/main/English_to_Hindi_translation_using_Tranasformer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers[torch] datasets evaluate sacrebleu sacremoses

In [ ]:
from datasets import load_dataset

raw_ds = load_dataset("cfilt/iitb-english-hindi", split='train[:2000]')

raw_ds = raw_ds.train_test_split(test_size=0.2)

print (raw_ds["train"][10])

# **Tokenization**

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
  input = [ex["en"] for ex in examples["translation"]]
  target = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(input, max_length=128, truncation=True)

  labels = tokenizer(target, max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


tokenized_ds = raw_ds.map(preprocess_function, batched=True)

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainer
raw_ds = load_dataset("cfilt/iitb-english-hindi", split='train[:2000]')

raw_ds = raw_ds.train_test_split(test_size=0.2)

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess_function(examples):
  input = [ex["en"] for ex in examples["translation"]]
  target = [ex["hi"] for ex in examples["translation"]]

  model_inputs = tokenizer(input, max_length=128, truncation=True)

  labels = tokenizer(target, max_length=128, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


tokenized_ds = raw_ds.map(preprocess_function, batched=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
output_dir="./translation_model",
learning_rate=2e-5,
per_device_train_batch_size=16,
weight_decay=0.01,
save_total_limit=1,
num_train_epochs=3,
predict_with_generate=True,
fp16=True,
report_to="none"
)
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_ds["train"],
eval_dataset=tokenized_ds["test"],
data_collator=data_collator,
)

In [ ]:
trainer.train()

In [29]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "Helsinki-NLP/opus-mt-en-hi"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

if 'model' not in globals():
    model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

def translate(text):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt").to(device)
    output_tokens = model.generate(**inputs)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

test_sentence = "Today we are studing about Data Science and I am very interested in this subject"
print(f"English: {test_sentence}")
print(f"Hindi: {translate(test_sentence)}")

English: Today we are studing about Data Science and I am very interested in this subject
Hindi: आज हम डाटा विज्ञान के बारे में पहुंचनी कर रहे हैं... ... और मैं इस य ी में बहुत रुचि रखता हूँ


In [30]:
import sacrebleu

predictions = []
references = []

# Use the test split from your dataset
for item in raw_ds["test"]:

    src_sentence = item["translation"]["en"]
    tgt_sentence = item["translation"]["hi"]

    pred_sentence = translate(src_sentence)

    predictions.append(pred_sentence)
    references.append(tgt_sentence)

# Calculate BLEU
bleu = sacrebleu.corpus_bleu(predictions, [references])

print("Final BLEU Score:", bleu.score)

Final BLEU Score: 32.38064854517944
